In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


# Import the libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
import plotly.express as px
#loading dataset
df = pd.read_csv("../input/avocado-prices/avocado.csv")

#Dropping the unamed column
df.drop('Unnamed: 0',inplace=True,axis=1)
df1=df
#checking the number of times the cities are repleated in Region column
dups = df.pivot_table(index = ['region'],aggfunc ='size')
print(dups)


In [ ]:
fig = px.box(df1, x='region', y='AveragePrice')
fig.show()

In [ ]:
# Picking One city
df = df.loc[df['region']=='SanFrancisco']
df['Date']=pd.to_datetime(df.Date)
df=df.sort_values(by="Date")

#Seperating Organic and Conventional Avacados.
df_org=df.loc[df['type']=='organic'].set_index('Date')
df_con=df.loc[df['type']=='conventional'].set_index('Date')


df_org.tail(10)

In [ ]:
#Plotting the Organic and Conventional Avacado price

plt.figure(figsize=(14,5))
sns.lineplot(data=df_org["AveragePrice"],label="Organic")
sns.lineplot(data=df_con["AveragePrice"],label="Conventional")
plt.xlabel("Date")
plt.title('Avocado prices')

In [ ]:
#Plotting Volume 

plt.figure(figsize=(14,5))

sns.lineplot(data=df_org["Total Volume"],label="Organic")
sns.lineplot(data=df_con["Total Volume"],label="Conventional")

plt.xlabel("Date")
plt.title(' Volume Avocado sold')

In [ ]:
#Random Forest Regressor
X = df1.drop('AveragePrice', axis=1)
y = df1['AveragePrice']



from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
len(X_train), len(X_test), len(y_train), len(y_test)


for label, content in df1.items():
    if pd.api.types.is_string_dtype(content):
        df1[label]  = pd.Categorical(content).codes+1

In [ ]:
#Model

y = df1["AveragePrice"]
X = df1.drop(columns = ["AveragePrice"])

X, y = shuffle(X, y, random_state = 7)
#  70% of the data will be used for train
#  30% will be used for testing

row_size = df1.shape[0]  # int 506
offset = int(row_size * 0.7) # int 354.2

# X_train = X[:] overfit olur 
X_train = X[:offset]  # Generating X_train data
X_test = X[offset:]   # Generating X_test data


y_train = y[:offset]  # Generating y_train data
y_test = y[offset:]  # yGenerating y_test data
model = RandomForestRegressor()

tuned = model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_pred

In [ ]:
score_r2 = metrics.r2_score(y_test, y_pred)
score_r2